* **Goal**: To built an AI agent using smolagents framework.
* **Agenda of this project**: To built an AI agent that searches top resturants which are nearby based on the users location and choice of cuisne. Then it generates the map and plots the routes to from user location to the fetched destinations.

**Install required packages**

In [1]:
!pip install geopy smolagents datasets langchain sentence-transformers faiss-cpu duckduckgo-search openai langchain-community --upgrade -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.

In [2]:
!pip install openrouteservice

**Import packages**

In [3]:
from typing import Optional
from smolagents import CodeAgent, HfApiModel, tool, DuckDuckGoSearchTool
import folium
import openrouteservice
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import requests
import time

**Login to HuggingFace**

In [5]:
from huggingface_hub import login

# Use your API token here
login("hf_token")

**Building AI agents that use code**
* I will be using codeagent template to define the tools.
*   Using open street map with folium for maps
*   Open route service for plotting the routes



In [12]:
# Your OpenRouteService API key
ORS_API_KEY = 'ORS_API_KEY'

client = openrouteservice.Client(key=ORS_API_KEY)

# Function to perform DuckDuckGo search
def duckduckgo_search(query: str) -> list:
    print(f"Performing DuckDuckGo search for query: {query}")
    response = requests.get("https://api.duckduckgo.com/", params={
        "q": query,
        "format": "json",
        "no_redirect": 1,
        "no_html": 1,
        "skip_disambig": 1
    })
    print(f"Received response: {response.status_code}")
    results = response.json().get("RelatedTopics", [])
    print(f"Search results: {results}")
    return results[:5]

@tool
def fetch_restaurants(location: str, cuisine: str) -> list:
    """Fetches the top 5 famous restaurants of a specific cuisine near the given location using DuckDuckGo search.

    Args:
        location: The location to search for restaurants.
        cuisine: The type of cuisine to search for.

    Returns:
        list: A list of dictionaries containing the restaurant names and locations.
    """
    search_query = f"top 5 {cuisine} restaurants near {location}"
    print(f"Fetching restaurants with query: {search_query}")
    search_results = duckduckgo_search(search_query)

    restaurants = []
    for result in search_results:
        title = result['Text']
        snippet = result.get('FirstURL', 'No address available')
        restaurants.append({"name": title, "location": snippet})
    print(f"Fetched restaurants: {restaurants}")
    return restaurants

@tool
def create_map(location: str, cuisine: str, restaurants: list) -> str:
    """Creates a map with routes to the top 5 famous restaurants of a specific cuisine near the given location.

    Args:
        location: The location to search for restaurants.
        cuisine: The type of cuisine to search for.
        restaurants: A list of dictionaries containing the restaurant names and locations.

    Returns:
        str: The HTML representation of the map.
    """
    print(f"Creating map for location: {location} with cuisine: {cuisine}")
    print(f"Restaurants data: {restaurants}")

    # Initialize the map centered on the given location
    geolocator = Nominatim(user_agent="myGeocoder", timeout=10)
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

    location_coords = None
    retries = 3
    for _ in range(retries):
        try:
            location_coords = geocode(location)
            if location_coords:
                break
        except Exception as e:
            print(f"Geocoding error for location {location}: {e}")
            time.sleep(2)

    if not location_coords:
        print("Failed to geocode the location.")
        return "Failed to geocode the location."

    m = folium.Map(location=[location_coords.latitude, location_coords.longitude], zoom_start=15)

    # Add a marker for the given location
    folium.Marker([location_coords.latitude, location_coords.longitude], tooltip=location, icon=folium.Icon(color='red')).add_to(m)

    # Add markers for the top 5 famous restaurants and plot routes
    for restaurant in restaurants:
        restaurant_coords = None
        for _ in range(retries):
            try:
                restaurant_coords = geocode(restaurant["location"])
                if restaurant_coords:
                    break
            except Exception as e:
                print(f"Geocoding error for restaurant {restaurant['name']}: {e}")
                time.sleep(2)

        if not restaurant_coords:
            print(f"Failed to geocode restaurant: {restaurant['name']}")
            continue

        folium.Marker([restaurant_coords.latitude, restaurant_coords.longitude], tooltip=restaurant["name"], icon=folium.Icon(color='blue')).add_to(m)

        # Get the route from the given location to the restaurant
        coords = [[location_coords.longitude, location_coords.latitude], [restaurant_coords.longitude, restaurant_coords.latitude]]
        try:
            route = client.directions(coordinates=coords, profile='foot-walking', format='geojson')
            folium.GeoJson(route['features'][0]['geometry']).add_to(m)
        except KeyError:
            print(f"Error: No route found for {restaurant['name']}. Response: {route}")

    # Save the map to an HTML string
    map_html = m._repr_html_()
    with open("map.html", "w") as file:
        file.write(map_html)

    print("Map saved to map.html")
    return "Map saved to map.html"

agent = CodeAgent(tools=[DuckDuckGoSearchTool(), fetch_restaurants, create_map], model=HfApiModel(), additional_authorized_imports=["datetime"])
agent.run("I am at times square, new york. Get me famous pizza restaurants and generate a map.")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I am at times square, new york. Get me famous pizza restaurants and generate a map.                             │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  restaurants = fetch_restaurants(location="Times Square, New York", cuisine="pizza")                              
  print(restaurants)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Fetching restaurants with query: top 5 pizza restaurants near Times Square, New York
Performing DuckDuckGo search for query: top 5 pizza restaurants near Times Square, New York
Received response: 202
Search results: []
Fetched restaurants: []


Execution logs:
[]

Out: None

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  restaurants = [                                                                                                  
      {"name": "Emmy Squared Pizza", "location": "311 W 48th St, New York, NY 10023"},                             
      {"name": "John's of Times Square", "location": "260 W 44th St, New York, NY 10036"},                         
      {"name": "Patzeria Perfect Pizza", "location": "126 W 45th St, New York, NY 10036"},                         
      {"name": "Carnegie Pizza", "location": "20 W 46th St, New York, NY 10036"},                                  
      {"name": "Carmine's - Time Square", "location": "133 W 44th St, New York, NY 10036"}                         
  ]                                                                                                                
                                                                                                                   
  map_url = create_map(location="Times Square, New York", cuisine="pizza", restaurants=restaurants)                
  final_answer(map_url)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Creating map for location: Times Square, New York with cuisine: pizza
Restaurants data: [{'name': 'Emmy Squared Pizza', 'location': '311 W 48th St, New York, NY 10023'}, {'name': "John's of Times Square", 'location': '260 W 44th St, New York, NY 10036'}, {'name': 'Patzeria Perfect Pizza', 'location': '126 W 45th St, New York, NY 10036'}, {'name': 'Carnegie Pizza', 'location': '20 W 46th St, New York, NY 10036'}, {'name': "Carmine's - Time Square", 'location': '133 W 44th St, New York, NY 10036'}]
Map saved to map.html


Out - Final answer: Map saved to map.html

[Step 4: Duration 27.95 seconds| Input tokens: 14,185 | Output tokens: 771]

'Map saved to map.html'

* **My prompt**: "I am at times square, new york. Get me famous pizza restaurants and generate a map."
* Based on the logs: AI agent used duck duck go to search web for the top pizza restaurants that are near times square, New york and was able to plot the map with routes successfully.

**Result**

In [13]:
from IPython.display import display, HTML

# Assuming you have already generated the map and saved it as "map.html"
# Read the HTML file
with open('map.html', 'r') as file:
    map_html = file.read()

# Display the HTML content
display(HTML(map_html))

Red marker indicates the origin and blue markers depict the 5 pizza restaurants and the routes plotted from the orign.

**Conclusion**
*   AI agent was able to complete the task successfully by thinking meticulously before acting and completing the task immaculately, achieveing the goal.


